In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler

from psotorch_lorenzo import PSO


def load_tensors(dataloc, shuffle=True, validation_split=0.5, random_seed=12345, phi=lambda x: x):
    data = np.loadtxt(dataloc)

    dataset_size = len(data)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    train_data = torch.from_numpy(data[train_indices]).float()
    val_data = torch.from_numpy(data[val_indices]).float()

    return phi(train_data[:, :2]), torch.squeeze(train_data[:, 2:]), phi(val_data[:, :2]), torch.squeeze(val_data[:, 2:])


def phi(X):
    squaredX = torch.pow(X, 2)
    return torch.hstack([X, squaredX])


class SingleLayerPerceptron(nn.Module):

    def __init__(self):
        super(SingleLayerPerceptron, self).__init__()
        self.fc = nn.Linear(6, 2)


    def forward(self, x):
        x = self.fc(x)
        return x


class SpiralClassifier(nn.Module):

    def __init__(self):
        super(SpiralClassifier, self).__init__()
        self.fc1 = nn.Linear(4, 6)
        self.fc2 = nn.Linear(6, 1)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"

device = torch.device(dev)

np.random.seed(12345)

num_epochs = 500
x_train, y_train, x_test, y_test = load_tensors('data/two_spirals.dat', device, phi=phi)

x_train = x_train.to(device)
y_train = y_train.to(device)

# model = SpiralClassifier().to(device)
model = SpiralClassifier().to(device)
loss = nn.BCEWithLogitsLoss().to(device)

optimizer = PSO(x_train, y_train, model=model, loss=loss, dim=35, inertia=0.7, a1=2.02, a2=2.02, population_size=30, time_steps=1000, search_range=1)

for i in range(num_epochs):

    if torch.cuda.is_available():
        y_train_preds = model(x_train.cuda())
        fitness = loss(y_train_preds, y_train.type(torch.FloatTensor).cuda().reshape(132,1))
    else:
        y_train_preds = model(x_train)
        fitness = loss(y_train_preds[:, 0], y_train.type(torch.FloatTensor))
    
    class_classified = [1 if i > 0.5 else 0 for i in torch.sigmoid(y_train_preds) ]
    accuracy = sum([1 for (i,j) in list(zip(y_train.type(torch.FloatTensor).cuda().reshape(132,1),class_classified))if i==j])/y_train_preds.shape[0]

    print(f"Epoch {i}; Acc={accuracy}")

    optimizer.step()

Epoch 0; Acc=0.5151515151515151
before update 0 tensor(657312.9375, device='cuda:0') 1e+30
after update 0 tensor(657312.9375, device='cuda:0') 657312.9375
before update 1 tensor(284842.4375, device='cuda:0') 1e+30
after update 1 tensor(284842.4375, device='cuda:0') 284842.4375
before update 2 tensor(384633.4062, device='cuda:0') 1e+30
after update 2 tensor(384633.4062, device='cuda:0') 384633.40625
before update 3 tensor(242181.8594, device='cuda:0') 1e+30
after update 3 tensor(242181.8594, device='cuda:0') 242181.859375
before update 4 tensor(375026.1250, device='cuda:0') 1e+30
after update 4 tensor(375026.1250, device='cuda:0') 375026.125
before update 5 tensor(392364.5938, device='cuda:0') 1e+30
after update 5 tensor(392364.5938, device='cuda:0') 392364.59375
before update 6 tensor(365656.5625, device='cuda:0') 1e+30
after update 6 tensor(365656.5625, device='cuda:0') 365656.5625
before update 7 tensor(837343.8125, device='cuda:0') 1e+30
after update 7 tensor(837343.8125, device='cu